In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import yfinance as yf
from langchain_core.tools import tool
import re
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Read the key
#api_key = os.getenv("OPENAI_API_KEY")

c:\Users\hp\anaconda3\envs\lanchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()

True

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
from langchain_core.tools import tool

from langchain_core.output_parsers import PydanticOutputParser
from typing import Literal
import logging
from numbers import Real
from dataclasses import dataclass
from enum import Enum

In [ ]:
import langchain
langchain.__version__

'1.2.0'

In [ ]:
from langchain.agents import create_agent, AgentState
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Literal
import logging

In [ ]:

def get_price_yfinance(symbol: str) -> float:
    """Fetch real-time price using yfinance (NSE: SUZLON.NS, NASDAQ: AAPL)."""
    try:
        ticker = yf.Ticker(symbol)
        price = ticker.history(period="1d")["Close"].iloc[-1]
        return float(price)
    except Exception as e:
        logging.getLogger(__name__).warning(f"yfinance failed for {symbol}: {e}")
        raise

def get_price_mock(symbol: str) -> float:
    """Mock prices for testing/agent development."""
    prices = {"SUZLON.NS": 52.59, "RELIANCE.NS": 2950.25, "AAPL": 225.15, "TSLA": 350.75}
    price = prices.get(symbol.upper())
    if not price:
        raise ValueError(f"No mock price for {symbol}")
    return price


In [ ]:


logger = logging.getLogger(__name__)

# Same portfolio tool (unchanged)
class PortfolioInput(BaseModel):
    symbol: str = Field(..., description="Stock symbol (e.g., SUZLON.NS)")
    shares: float = Field(..., gt=0)
    cost_per_share: float = Field(..., gt=0)
    source: Literal["yfinance", "mock"] = "yfinance"



@tool(args_schema=PortfolioInput, return_direct=True)
def compute_portfolio_metrics(symbol: str, shares: float, cost_per_share: float,
                            source: str = "yfinance") -> str:
    """
    Compute portfolio metrics for any stock symbol.

    Perfect for investment analysis agents. Returns formatted results.
    """
    # Implementation same as before
    symbol = symbol.upper().strip()
    price_funcs = {"yfinance": get_price_yfinance, "mock": get_price_mock}
    price = price_funcs.get(source, get_price_yfinance)(symbol)

    total_value = shares * price
    total_cost = shares * cost_per_share
    profit = total_value - total_cost
    roi = (profit / total_cost * 100) if total_cost > 0 else 0

    return (f"📊 **{symbol}**: Price ₹{price:,.2f} | Value ₹{total_value:,.2f} "
            f"| Cost ₹{total_cost:,.2f} | Profit ₹{profit:,.2f} | ROI {roi:.2f}%")

# 1.2 Agent Creation
llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [compute_portfolio_metrics]

agent = create_agent(
    model=llm,
    tools=tools,
    # Optional: Custom state extending AgentState
    state_schema=None  # or custom AgentState subclass
)

# Direct invocation (no AgentExecutor needed)


In [ ]:
result = agent.invoke(
    {"messages": [HumanMessage(content=""" Analyze 1000 Reliance Power shares at ₹45 """)]},
    # Optional context for stateful agents
    context={"user_id": "portfolio_user_123"}
)

print(result["messages"][-1].content)


📊 **RPOWER.NS**: Price ₹38.56 | Value ₹38,560.00 | Cost ₹45,000.00 | Profit ₹-6,440.00 | ROI -14.31%
